In [91]:
import pandas as pd

In [92]:
nevada_raw_df = pd.read_csv('Nevada_Report_raw.csv')
nevada_raw_df = nevada_raw_df.drop(nevada_raw_df.columns[0], axis=1)
nevada_raw_df = nevada_raw_df[(nevada_raw_df[nevada_raw_df.columns[0]] != 'Invisible Institute Request') & (nevada_raw_df[nevada_raw_df.columns[0]] != '03:50PM Certification By Agency') & (nevada_raw_df[nevada_raw_df.columns[0]] != 'Certificate Level Status Date Status Cert # Certified Expires Probation') & (~nevada_raw_df[nevada_raw_df.columns[0]].str.contains('AUG 21,2019 Nevada Commission on POST Page:'))]
nevada_raw_df = nevada_raw_df.drop([56434,56435,56436])
nevada_raw_df

,0
4,11th District
5,"Baldini, Daniel C. 18994"
6,Basic: Category II 7-01-2015 Active 9-22-2000
7,"Banfield, Justin E. 33603"
8,Basic: Category I 10-30-2017 Active 11-10-2016...
...,...
56425,"Wilmoth, Monte R. 22970"
56426,Basic: Category I 1-14-2011 Inactive 10-16-198...
56427,"Worden, William D. 14436"
56428,Basic: Category I 3-03-2014 Inactive 5-05-1996...


In [93]:
nevada_raw = nevada_raw_df.values.tolist()
nevada_raw

[['11th District'],
 ['Baldini, Daniel C. 18994'],
 ['Basic: Category II 7-01-2015 Active 9-22-2000'],
 ['Banfield, Justin E. 33603'],
 ['Basic: Category I 10-30-2017 Active 11-10-2016 2-16-2017'],
 ['Castellanos, Noliena F. 24970'],
 ['Basic: Category I 7-01-2015 Active 5-19-2006'],
 ['McKinney, Damien W. 31904'],
 ['Basic: Category II 8-01-2016 Active 4-01-2014'],
 ['North, Kathryn R. 29912'],
 ['Basic: Category II 7-01-2015 Active 5-19-2011'],
 ['Schlepp, Curtis J. 23288'],
 ['Basic: Category I 2-13-2018 Active 11-03-1996'],
 ['Basic: Category III 2-13-2018 Active 3-04-1995'],
 ['Tippens, Craig L. 14662'],
 ['Basic: Category II 6-08-2018 Inactive 9-22-2000 6-08-2023'],
 ['Employees this Agency: 7'],
 ['1st Judicial Dist Ct, Dept 1'],
 ['Warne, James R. 18093'],
 ['Basic: Category I 7-01-2017 Inactive 12-23-1977 7-01-2022'],
 ['Employees this Agency: 1'],
 ['3rd District'],
 ['Rye, Cherie C. 14445'],
 ['Basic: Category I 8-04-2015 Active 8-08-1997'],
 ['Employees this Agency: 1'],
 [

In [94]:
def is_district_header(row):
    return (len(row) == 1) and (not row[0][-4:].isdigit()) and ('Employees' not in row[0])

def is_district_end(row):
    return (len(row) == 1) and (not row[0][-4:].isdigit()) and ('Employees' in row[0])

def is_name_header(row):
    return (len(row) == 1) and (row[0][-5:].isdigit())

def add_none(row):
    # adds None appropriately depending on whether last date is 'Expires' or 'Probation'
    cert_yr, aft_yr = int(row[-2][-4:]), int(row[-1][-4:])
    if (aft_yr-cert_yr) <= 2:
        row.insert(3, None)
    else:
        row.append(None)
    return row

In [95]:
for row in nevada_raw:
    if is_district_header(row):
        print(row)

['11th District']
['1st Judicial Dist Ct, Dept 1']
['3rd District']
['4th District, Dept. II']
['4th District, Elko Co Juvenile Probation']
['5th District']
['6th District, Juvenile Probation']
['7th District']
['8th Judicial District Court Marshals Div']
['Boulder City Municipal Court']
['Boulder City PD']
['Boulder City Township Justice Court']
["Boulder Township Constable's Office"]
['Carlin PD']
['Carson City DA']
['Carson City Dept of Alt Sentencing']
['Carson City Justice/Municipal Court']
['Carson City Juvenile Probation Services']
['Carson City Parks & Recreation']
['Carson City SO']
['Carson City School Dist Safety Enforc']
['Churchill Co Juvenile Justice Center']
['Churchill Co SO']
['City of LV Dept of Public Safety']
['Clark Co DA']
['Clark Co Dept of Juvenile Justice Svcs']
['Clark Co FD']
['Clark Co Park Police']
['Clark Co School District PD']
['College of Southern NV PD']
['Dayton Justice Court']
['Douglas Co DA']
['Douglas Co Dept of Alt Sentencing']
['Douglas Co Juven

In [99]:
nevada_final = []

district,name,level = '','',''
certificate,vars = '',[]
for row in nevada_raw:
    row[0] = row[0].replace('Basic ','')
    if is_district_end(row):
        continue
    if is_district_header(row):
        district = row[0]
        continue
    if is_name_header(row):
        name, level = row[0][:-5].strip(), row[0][-5:]
        continue
    for i in range(len(row[0])-1, 1, -1):
        if row[0][i] == ' ' and (row[0][i-1] == 'I' or row[0][i-7:i] == 'Reserve'):
            certificate, vars = row[0][:i].strip(), row[0][i:].strip().split(' ')
            break
    if len(vars) == 5 or len(vars) == 3:
        nevada_final.append([district,name,level,certificate]+vars)
    else:
        print(row)
        vars = add_none(vars)
        nevada_final.append([district,name,level,certificate]+vars)

['Basic: Category I 10-30-2017 Active 11-10-2016 2-16-2017']
['Basic: Category II 6-08-2018 Inactive 9-22-2000 6-08-2023']
['Basic: Category I 7-01-2017 Inactive 12-23-1977 7-01-2022']
['Basic: Category I 8-19-2019 Inactive 11-08-1984 8-19-2024']
['Basic: Category II 12-05-2016 Active 3-31-2016 12-05-2016']
['Basic: Category I 2-13-2015 Inactive 8-25-1994 2-13-2020']
['Basic: Category II 3-29-2019 Inactive 11-10-2011 3-29-2024']
['Basic: Category II 12-29-2015 Inactive 11-16-2007 12-29-2020']
['Basic: Category I 3-04-2019 Inactive 3-03-1996 3-04-2024']
['Basic: Category II 4-21-2011 Inactive 5-21-2004 4-21-2016']
['Basic: Category I 7-05-2016 Active 11-14-2013 11-14-2013']
['Basic: Category I 4-17-2017 Inactive 7-03-2001 4-17-2022']
['Basic: Category II 5-29-2018 Inactive 10-11-2012 5-29-2023']
['Basic: Category II 3-25-2011 Inactive 12-17-2003 3-25-2016']
['Basic: Category II 12-05-2016 Active 3-31-2016 12-05-2016']
['Basic: Category II 2-15-2016 Inactive 11-22-2002 2-15-2021']
['Basi

In [100]:
nevada_final_df = pd.DataFrame(nevada_final, columns=['Agency', 'Name', 'Level', 'Certificate', 'Status Date', 'Status', 'Certified', 'Expires', 'Probation'])
nevada_final_df

,Agency,Name,Level,Certificate,Status Date,Status,Certified,Expires,Probation
0,11th District,"Baldini, Daniel C.",18994,Basic: Category II,7-01-2015,Active,9-22-2000,None,None
1,11th District,"Banfield, Justin E.",33603,Basic: Category I,10-30-2017,Active,11-10-2016,None,2-16-2017
2,11th District,"Castellanos, Noliena F.",24970,Basic: Category I,7-01-2015,Active,5-19-2006,None,None
3,11th District,"McKinney, Damien W.",31904,Basic: Category II,8-01-2016,Active,4-01-2014,None,None
4,11th District,"North, Kathryn R.",29912,Basic: Category II,7-01-2015,Active,5-19-2011,None,None
...,...,...,...,...,...,...,...,...,...
27548,Yomba Shoshone PD,"Musgrave, Thomas B.",12955,Basic: Category III,8-04-2016,Inactive,8-21-1993,8-04-2021,None
27549,Yomba Shoshone PD,"Parsons, Christopher T.",14013,Basic: Category I,2-11-2015,Active,10-23-1987,None,None
27550,Yomba Shoshone PD,"Perry, Earl W.",27308,Basic: Category I,7-24-2017,Active,5-16-2008,None,None
27551,Yomba Shoshone PD,"Wilmoth, Monte R.",22970,Basic: Category I,1-14-2011,Inactive,10-16-1981,1-14-2016,None


In [101]:
nevada_final_df.to_csv('Nevada_Report.csv')
print('done')

done
